In [1]:
%cd ..

/Users/ext-elias.melas/Documents/Gitcode/ar_finetune


In [2]:
import pandas as pd
from openai import OpenAI, AsyncClient
from pydantic import BaseModel, Field
from typing import Optional, List, Dict, Any
from enum import StrEnum
import asyncio
import json

from dotenv import load_dotenv
load_dotenv()

client = AsyncClient()

In [3]:
# set pd rows to none
pd.set_option('display.max_columns', None)

In [4]:
def lambda_sort_by_message_order(message_array):
    data = json.loads(message_array)
    return sorted(data, key=lambda obj: int(obj['message_order']))

In [5]:
import pandas as pd

features = ['accounts',
'MaxDevicesRule',
'PhiRejectionsForSameDeviceRule',
'IpMismatchRule',
'PhiRejectionsForSameTargetAccountRule',
'RelatedAccountsPerDeviceRule',
'SameIpAndDeviceNameSearchRequestsRule',
'LocationAndDeviceChangeRule',
'SuspiciousSessionsRule',
'OverlappingMismatchingSessionsRule',
'BannedAccountRule',
'CommonDevicesRule',
'PhishOrScamInSupportNotesRule',
'RecentSessionsRule',
'SameRequesterAndTargetAccountsRule',
'ScidBoundThresholdRule',
'TotalSpendRule',
'YoungPlayerRule']

# df_messages = pd.read_csv('data/verification/model_messages_sep_2025_stratified.csv')
df_messages = pd.read_csv('data/verification/failed_hs_bots_agents_messages_30d_all_games.csv')
df_messages['messages_full'] = df_messages['message_map_list'].apply(lambda_sort_by_message_order)
df_messages['messages_player_only'] = df_messages['messages_full'].apply(
    lambda x: ',\n'.join(
        f"{i}. {m['body']}"
        for i, m in enumerate((m for m in x if m['role'] == 'player'), start=1)
    )
)
df_messages['agent_notes'] = df_messages['messages_full'].apply(
    lambda x: '\n'.join(
        f"{i}. {m['body']}"
        for i, m in enumerate(
            (m for m in x if m.get('message_type') == 'add_private_note' and m.get('role') == 'agent'),
            start=1
        )
    )
)
df_ar_results = pd.read_csv('data/verification/ar_sep_2025.csv')

# df_models_messages_features = df_messages.merge(df_model_features[['externalId', 'ml_features']], left_on='ticket_id', right_on='externalId', how='left')
df_models_messages_features = df_messages.merge(df_ar_results, right_on='ticket_id', left_on='ticket_id', how='outer')

In [10]:
df_models_messages_features = df_models_messages_features.loc[~df_models_messages_features['features_messages'].isna()]

In [11]:
def get_phishing_attempts_either(row):
    if  row['phishing_attempts_agent'] == 1 or row['phishing_attempts_bot'] == 1:
        return True
    else:
        return None

df_models_messages_features['phishing_attempts_either'] = df_models_messages_features.apply(lambda x: get_phishing_attempts_either(x), axis=1).fillna(False)

/var/folders/0c/96px_5vs3wj8lpnj4_mzxvxm0000gq/T/ipykernel_9480/2644108008.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_models_messages_features['phishing_attempts_either'] = df_models_messages_features.apply(lambda x: get_phishing_attempts_either(x), axis=1).fillna(False)


In [12]:
import re

def get_features_from_messages(x):
    substring = 'Risk assessment results for malicious_account_recovery:'
    try:
        for m in x:
            body = m['body']
            if 'Risk assessment results' in body:
                return body.split(substring)[1]
    except Exception as e:
        # print(e)
        return None

def get_ml_result_from_messages(x):
    pattern = r'account_recovery_eligibility:\s*([A-Z_]+)(?=\s*✅?\s*SuspiciousSessionsRule)'
    try:
        for m in x:
            body = m.get('body', '')
            if 'SuspiciousSessionsRule' in body:
                m_ = re.search(pattern, body)
                return m_.group(1) if m_ else None
        return None
    except Exception as e:
        # print(e)
        return None


def get_ml_features_from_messages(x):
    pattern = r'SuspiciousSessionsRule[\s\S]*?(?=\s+Compare Tab:)'
    try:
        for m in x:
            body = m.get('body', '')
            if 'SuspiciousSessionsRule' in body:
                m_ = re.search(pattern, body)
                return m_.group(0) if m_ else None
        return None
    except Exception as e:
        # print(e)
        return None

df_models_messages_features['features_messages'] = df_models_messages_features['messages_full'].apply(lambda x: get_features_from_messages(x))  
df_models_messages_features['ml_features_messages'] = df_models_messages_features['messages_full'].apply(lambda x: get_ml_features_from_messages(x))
df_models_messages_features['ml_result_messages'] = df_models_messages_features['messages_full'].apply(lambda x: get_ml_result_from_messages(x))

# full missing values with ml_features_messages column
# df_models_messages_features['ml_features_messages'].fillna(df_models_messages_features['ml_features'],inplace=True)

# s = df_models_messages_features['messages_full'].iloc[0]
display(
    df_models_messages_features['features_messages'].value_counts(dropna=False),
    df_models_messages_features['ml_features_messages'].value_counts(dropna=False),
    df_models_messages_features['ml_result_messages'].value_counts(dropna=False)
)

features_messages
Clash Royale / 10-53051687 / PASS  ✅ Rule 1: 0 during last 60 days (Yellow 3 / Red 6) ✅ Rule 2: [True] ✅ Rule 3: 0 during last 1 day (Yellow 2 / Red 3) ✅ Rule 4: Requester 1 during last 7 days (Yellow 3 / Red 5) ✅ Rule 5: at least 3 in days 7 (Yellow 10 / Red 15) ✅ Rule 6: 0 during last 7 days (Yellow 16 / Red 21) ✅ Rule 7: Requester account had 0 simultaneous change(s) of location and device during the last 7 days. ✅ Rule 8: Requester account has no suspicious sessions      8
Clash Royale / 54-44710726 / PASS  ✅ Rule 1: 0 during last 60 days (Yellow 3 / Red 6) ✅ Rule 2: [True] ✅ Rule 3: 0 during last 1 day (Yellow 2 / Red 3) ✅ Rule 4: Requester 1 during last 7 days (Yellow 3 / Red 5) ✅ Rule 5: at least 0 in days 7 (Yellow 10 / Red 15) ✅ Rule 6: 0 during last 7 days (Yellow 16 / Red 21) ✅ Rule 7: Requester account had 0 simultaneous change(s) of location and device during the last 7 days. ✅ Rule 8: Requester account has no suspicious sessions      8
Brawl Stars / 9-11

ml_features_messages
None                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

ml_result_messages
ASSIGN_TO_AGENT           70459
None                      22352
NORMAL_AUTO_RECOVERY       1539
YP_CONVERSION_RECOVERY      352
YP_YP_RECOVERY              212
AUTO_REJECTION                2
Name: count, dtype: int64

In [13]:
training_data_list = []

for i,row in df_models_messages_features.iterrows():
    training_data = {
        'messages_full': row['messages_full'],
        'ticket_id': row['ticket_id'],
        'messages_player': row['messages_player_only'],
        'ticket_tags_with_description': row['ticket_tags_with_description'],
        'is_phishing': row['phishing_attempts_either'],
        'features': row['features_messages'],
        "ml_features": row['ml_features_messages'],
        "ml_result": row['ml_result_messages'],
        'agent_notes': row['agent_notes']
    }
    training_data_list.append(training_data)

lora_data = pd.DataFrame(training_data_list)

In [14]:
display(lora_data.head(),lora_data.shape)

,messages_full,ticket_id,messages_player,ticket_tags_with_description,is_phishing,features,ml_features,ml_result,agent_notes
0,"[{'message_order': '1', 'role': 'player', 'bod...",340843427,"1. Hello!,\n2. Account access and Supercell ID...","[{""ticket_tag"":""ar-inconclusive"",""tag_descript...",False,Clash of Clans / 87-49646810 / PASS ✅ Rule 1...,None,None,1. https://pandora.supercellsupport.com/clasho...
1,"[{'message_order': '1', 'role': 'player', 'bod...",340843451,"1. Hello!,\n2. <strong>Form Input Response</st...","[{""ticket_tag"":""attachment-message redaction"",...",False,Clash of Clans / 59-52182461 / PASS ✅ Rule 1...,SuspiciousSessionsRule: Target account has no ...,ASSIGN_TO_AGENT,1. Issue: Found TA TA link: https://pandora.su...
2,"[{'message_order': '1', 'role': 'player', 'bod...",340843598,"1. Hello!,\n2. <strong>Form Input Response</st...","[{""ticket_tag"":""ar-lost-recent access"",""tag_de...",False,Clash of Clans / 30-51012110 / PASS ✅ Rule 1...,SuspiciousSessionsRule: Target account has no ...,ASSIGN_TO_AGENT,1. Issue: AR Investigation: RA is not TA RA = ...
3,"[{'message_order': '1', 'role': 'player', 'bod...",340843619,"1. Hello!,\n2. Purchases,\n3. See All Topics,\...","[{""ticket_tag"":""ar-lost-recovery"",""tag_descrip...",False,Clash of Clans / 59-24576690 / PASS ✅ Rule 1...,SuspiciousSessionsRule: Target account has no ...,ASSIGN_TO_AGENT,1. https://pandora.supercellsupport.com/clasho...
4,"[{'message_order': '1', 'role': 'player', 'bod...",340843634,"1. i want to cancel my account deleted,\n2. Re...","[{""ticket_tag"":""gdpr-deletion-cancelled"",""tag_...",False,Clash Royale / 18-49017848 / PASS ✅ Rule 1: ...,None,None,1. Cancelled scheduled deletion. Internal reas...


(94916, 9)

In [15]:
lora_data['is_phishing'].value_counts(dropna=False)

is_phishing
False    93122
True      1794
Name: count, dtype: int64

In [16]:
import pandas as pd

true_df  = lora_data[lora_data["is_phishing"] == True]
false_df = lora_data[lora_data["is_phishing"] == False]

false_down = false_df.sample(
    n=len(true_df),
    random_state=42
)

lora_data_balanced = (
    pd.concat([true_df, false_down])
      .sample(frac=1, random_state=42)  # shuffle
      .reset_index(drop=True)
)

lora_data_balanced["is_phishing"].value_counts()


is_phishing
False    1794
True     1794
Name: count, dtype: int64

In [17]:
from sklearn.model_selection import train_test_split

# stratified split into train and test
lora_train, lora_test = train_test_split(lora_data_balanced, test_size=0.2, random_state=42, stratify=lora_data_balanced['is_phishing'])
display(lora_train['is_phishing'].value_counts(dropna=False))
display(lora_test['is_phishing'].value_counts(dropna=False))


is_phishing
False    1435
True     1435
Name: count, dtype: int64

is_phishing
False    359
True     359
Name: count, dtype: int64

In [18]:
class VerificationCategory(StrEnum):
    GENUINE = "GENUINE"
    PHISHING = "PHISHING"
    ASSIGN_TO_AGENT = "ASSIGN_TO_AGENT"

class VerificationClassification(BaseModel):
    category: VerificationCategory = Field(..., description="The category of the verification. Possible values are 'GENUINE', 'PHISHING', or 'ASSIGN_TO_AGENT' if not sure.")
    reason: str = Field(..., description="The reason for the category classification")
    reason_keyword: str = Field(..., description="A few keywords that describes the reason, in snake case")
    source: List[str] = Field(default_factory=list, description="A list of reasons for the intent, with an example from the conversation")

policy_doc = open('data/verification/phishing_policy_doc.md', 'r').read()

system_prompt = f"""
You are a support engineer in a mobile gaming company. Your task is to classify if a chat log belongs to a phishing attempt or not.

If the chat log is a phishing attempt, return "PHISHING" for category, the reason for the phishing attempt, the reason keyword and the source.
If the chat log is not a phishing attempt, return "GENUINE" for category, an empty string for reason, an empty string for reason_keyword and an empty list for source.
If unclear, return "ASSIGN_TO_AGENT" for category.

You can use the following information to help you classify the chat log:
{policy_doc}

"""

async def classify_phishing(
    messages: str,
    features: str, 
    ml_features: str,
    system_prompt: str = system_prompt
    ) -> VerificationClassification:

    user_prompt = f"""
    Message Chat log:
    ---------------------------------------------------
    \n{messages}\n\n
    ---------------------------------------------------
    Compromised account analysis:
    ---------------------------------------------------
    \n{features}\n\n
    ---------------------------------------------------

    Machine learning features:
    ---------------------------------------------------
    \n{ml_features}\n\n
    ---------------------------------------------------

    """

    # print(user_prompt)

    response = await client.responses.parse(
        # model="o4-mini-2025-04-16",
        # model = 'o3-2025-04-16',
        model = 'gpt-5.2-2025-12-11',
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        text_format=VerificationClassification,
    )

    return response.output_parsed

async def process_all_messages_phishing(df_messages: pd.DataFrame) -> List[Dict[str, Any]]:

    """Process all messages with their ticket IDs asynchronously"""
    semaphore = asyncio.Semaphore(100)  # Limit concurrency

    async def process_single_message(
        ticket_id: Any, 
        message: str, 
        tag_description: str,
        features: str,
        ml_features: str,
        ) -> Dict[str, Any]:
        async with semaphore:
            intent_result = await classify_phishing(messages=message, features=features, ml_features=ml_features)
            return {
                'ticket_id': ticket_id,
                'message': message,
                'ticket_tags': tag_description,
                'features': features,
                'ml_features': ml_features,
                'category': intent_result.category if intent_result.category else None,
                'reason': intent_result.reason if intent_result.reason else None,
                'reason_keyword': intent_result.reason_keyword if intent_result.reason_keyword else None,
                # 'intent_list': [intent.value for intent in intent_result.intent],
                'source': intent_result.source,
                'success': True,
                'error': None
        }

    tasks = [
        process_single_message(row['ticket_id'], row['messages_player'], row['ticket_tags_with_description'], row['features'], row['ml_features'])
        for _, row in df_messages.iterrows()
    ]
    results = await asyncio.gather(*tasks)
    return results

phishing_results = await process_all_messages_phishing(lora_test)

df_phishing = pd.DataFrame(phishing_results)
df_phishing_messages = lora_test.merge(
    df_phishing[
        ["ticket_id", "category", "reason", "reason_keyword", "source", "success", "error",]
    ],
    on="ticket_id",
    how="left"
)

In [ ]:
# df_phishing_messages.to_parquet('data/verification/phishing_classifier_results.parquet')
df_phishing_messages = pd.read_parquet('data/verification/phishing_classifier_results.parquet')

In [20]:
# df_phishing_messages.loc[~df_phishing_messages['category'].isna()]
def create_llm_label(x):
    is_phishing = x['is_phishing']
    llm_result = x['category']
    if llm_result == 'GENUINE':
        return False
    else:
        return True

df_phishing_messages['is_phishing_llm'] = df_phishing_messages.apply(create_llm_label, axis=1)
display(
    df_phishing_messages['is_phishing_llm'].value_counts(dropna=False),
    df_phishing_messages['category'].value_counts(dropna=False)
    )

is_phishing_llm
True     632
False     90
Name: count, dtype: int64

category
PHISHING           349
ASSIGN_TO_AGENT    283
GENUINE             90
Name: count, dtype: int64

In [21]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

y_true = df_phishing_messages["is_phishing"]
y_pred = df_phishing_messages["is_phishing_llm"]

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
# [[TN, FP],
#  [FN, TP]]

print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
print(f"Confusion Matrix: {cm}")

Precision: 0.5332, Recall: 0.9284, F1 Score: 0.6774
Confusion Matrix: [[ 64 295]
 [ 26 337]]


### Preparing FT Training Data

In [22]:
lora_train

,messages_full,ticket_id,messages_player,ticket_tags_with_description,is_phishing,features,ml_features,ml_result,agent_notes
2417,"[{'message_order': '1', 'role': 'player', 'bod...",342277410,"1. Olá!,\n2. Account access and Supercell ID,\...","[{""ticket_tag"":""ar-lost-recovery"",""tag_descrip...",False,mo.co / 1-2705795 / PASS ✅ Rule 1: 1 during ...,SuspiciousSessionsRule: Target account has no ...,ASSIGN_TO_AGENT,1. https://pandora.supercellsupport.com/moco/a...
1678,"[{'message_order': '1', 'role': 'player', 'bod...",341673145,"1. Hello!,\n2. <strong>Form Input Response</st...","[{""ticket_tag"":""ar-phisher"",""tag_description"":...",True,Clash of Clans / 21-50741604 / PASS ✅ Rule 1...,SuspiciousSessionsRule: Target account has no ...,ASSIGN_TO_AGENT,1. https://pandora.supercellsupport.com/clasho...
2634,"[{'message_order': '1', 'role': 'player', 'bod...",341728051,"1. Dear Support, My account is blocked on my d...","[{""ticket_tag"":""ar-lost-recovery"",""tag_descrip...",False,Clash Royale / 37-81834091 / PASS ✅ Rule 1: ...,SuspiciousSessionsRule: Target account has no ...,ASSIGN_TO_AGENT,1. https://pandora.supercellsupport.com/clashr...
3273,"[{'message_order': '1', 'role': 'player', 'bod...",341922759,"1. Hello!,\n2. Account access and Supercell ID...","[{""ticket_tag"":""ar-phisher"",""tag_description"":...",True,Clash of Clans / 10-25462793 / PASS ✅ Rule 1...,SuspiciousSessionsRule: Target account has no ...,ASSIGN_TO_AGENT,1. Issue RA wants to recover TA Investigation ...
1768,"[{'message_order': '1', 'role': 'automation', ...",341584370,"1. A different game account,\n2. <strong>Form ...","[{""ticket_tag"":""ar-lost-inquiry"",""tag_descript...",False,Hay Day / 22-40737551 / PASS ✅ Rule 1: 0 dur...,None,None,1. ta https://pandora.supercellsupport.com/hay...
...,...,...,...,...,...,...,...,...,...
129,"[{'message_order': '1', 'role': 'automation', ...",341484952,"1. I cannot make any purchases in-game,\n2. En...","[{""ticket_tag"":""ar-lost-inquiry"",""tag_descript...",False,Clash Royale / 38-68503159 / PASS ✅ Rule 1: ...,None,None,1. Two requests. Will start with the first one...
1692,"[{'message_order': '1', 'role': 'player', 'bod...",342465173,"1. Hello!,\n2. <strong>Form Input Response</st...","[{""ticket_tag"":""ar-lost-recovery"",""tag_descrip...",False,Clash of Clans / 46-49998904 / PASS ✅ Rule 1...,SuspiciousSessionsRule: Target account has no ...,ASSIGN_TO_AGENT,1. Issue: Lost account request TA: https://pa...
2981,"[{'message_order': '1', 'role': 'player', 'bod...",343015635,"1. Hello!,\n2. Account access and Supercell ID...","[{""ticket_tag"":""aut-age gate"",""tag_description...",True,Clash of Clans / 30-50005952 / PASS ✅ Rule 1...,SuspiciousSessionsRule: Target account has no ...,ASSIGN_TO_AGENT,1. Issue CA https://pandora.supercellsupport.c...
1159,"[{'message_order': '1', 'role': 'automation', ...",341733941,"1. A different game account,\n2. <strong>Form ...","[{""ticket_tag"":""ar-phisher"",""tag_description"":...",False,Hay Day / 7-43864967 / PASS ✅ Rule 1: 0 duri...,SuspiciousSessionsRule: Target account has no ...,ASSIGN_TO_AGENT,1. https://pandora.supercellsupport.com/hayday...


In [ ]:
system_prompt = open('data/verification/system_prompt.md', 'r').read()

ft_training_data_list = []
for i,d in lora_train.iterrows():
    messages = d['messages_player']
    features = d['features']
    ml_features = d['ml_features']
    is_phishing = d['is_phishing']
    
    user_prompt = f"""
    Chat log (player messages only):
    ---------------------------------------------------
    \n{messages}\n\n
    ---------------------------------------------------
    Compromised account analysis:
    ---------------------------------------------------
    \n{features}\n\n
    ---------------------------------------------------

    Machine learning features:
    ---------------------------------------------------
    \n{ml_features}\n\n
    ---------------------------------------------------

    """

    input = """
    {system_prompt}
    {user_prompt}
    """

    output = is_phishing

    ft_training_data = {
        'instruction': system_prompt,
        'input': user_prompt,
        'output': f'is_phishing: {is_phishing}, agent_notes: {d["agent_notes"]}'
    }
    ft_training_data_list.append(ft_training_data)


In [ ]:
ft_training_data_list[0]

# with open("data/finetuning/training_data.json", "w", encoding="utf-8") as f:
#     json.dump(ft_training_data_list, f, indent=2, ensure_ascii=False)

{'instruction': 'You are a support engineer in a mobile gaming company. Your task is to classify if a chat log belongs to a phishing attempt or not.\n\nIf the chat log is a phishing attempt, return "PHISHING" for category, the reason for the phishing attempt, the reason keyword and the source.\nIf the chat log is not a phishing attempt, return "GENUINE" for category, an empty string for reason, an empty string for reason_keyword and an empty list for source.\nIf unclear, return "ASSIGN_TO_AGENT" for category.\n',
 'input': "\n    Message Chat log:\n    ---------------------------------------------------\n    \n1. Olá!,\n2. Account access and Supercell ID,\n3. Continue,\n4. <strong>Form Input Response</strong><br/><br/>What is the player tag of your lost account? (#ABC1234)?<br/><strong>LQ2PRL28</strong><br/><br/>What is the name of your lost account?<br/><strong>claralopesaf</strong><br/><br/>What is the trophy count (0-15000) of your lost account?<br/><strong>0</strong><br/><br/>What 

# LoRA

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# model_name = "meta-llama/Llama-3.1-8B"
model_name = "./models/llama-3.1-8b"

/Users/ext-elias.melas/Documents/Gitcode/ar_finetune/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [23]:
# Load model for training.
# Note: `device_map="auto"` is for inference/sharding and can break `Trainer` on MPS (it can lead to `torch.device(None)` inside accelerate).

device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=(torch.float16 if device in {"cuda", "mps"} else torch.float32),
)
model.to(device)

`torch_dtype` is deprecated! Use `dtype` instead!


Using device: mps


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.19s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
  

### (Or QLoRA)

In [24]:
from transformers import BitsAndBytesConfig

# 4-bit QLoRA via bitsandbytes requires CUDA. On macOS/MPS it won’t work.
if not torch.cuda.is_available():
    print("Skipping 4-bit QLoRA (bitsandbytes requires CUDA).")
else:
    # 4-bit quantization config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
    )

Skipping 4-bit QLoRA (bitsandbytes requires CUDA).


In [25]:
from peft import LoraConfig, get_peft_model

In [26]:
lora_config = LoraConfig(
    r=16,                          # Rank
    lora_alpha=32,                 # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Which layers
    lora_dropout=0.05,             # Dropout for regularization
    bias="none",                   # Don't train bias
    task_type="CAUSAL_LM"          # Language modeling task
)
# Apply LoRA to model
model = get_peft_model(model, lora_config)
# Check trainable parameters
model.print_trainable_parameters()
# Output: trainable params: 4,194,304 || all params: 8,030,261,248 || trainable%: 0.05%

trainable params: 6,815,744 || all params: 8,037,076,992 || trainable%: 0.0848


### Prepare training data

In [27]:
from datasets import load_dataset

# Load your dataset
dataset = load_dataset("json", data_files="data/finetuning/training_data.json")
# Tokenize
def format_instruction(example):
    text = f"### Instruction:\n{example['instruction']}\n\n"
    if example.get('input'):
        text += f"### Input:\n{example['input']}\n\n"
    text += f"### Response:\n{example['output']}"
    return {"text": text}
dataset = dataset.map(format_instruction)
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        max_length=2048,
        padding="max_length"
    )
tokenized_dataset = dataset.map(tokenize, batched=True)


Generating train split: 2330 examples [00:00, 24107.57 examples/s]
Map: 100%|██████████| 2330/2330 [00:01<00:00, 1603.01 examples/s]


### Set training arguments

In [28]:
from transformers import TrainingArguments

# bitsandbytes optimizer only works on CUDA; on MPS/CPU use a standard optimizer.
use_cuda = torch.cuda.is_available()

training_args = TrainingArguments(
    output_dir="./llama-lora-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=1 if not use_cuda else 4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True if use_cuda else False,
    bf16=False,
    save_strategy="epoch",
    logging_steps=10,
    warmup_steps=100,
    optim="paged_adamw_8bit" if use_cuda else "adamw_torch",
)

### Train model

In [29]:
from transformers import Trainer, DataCollatorForLanguageModeling

# For causal LM fine-tuning, make sure we produce `labels`.
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
)

# Start training
# trainer.train()


getting runtime error with LoRA:

RuntimeError: MPS backend out of memory (MPS allocated: 77.76 GiB, other allocations: 2.13 GiB, max allowed: 81.60 GiB). Tried to allocate 2.00 GiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).


In [30]:
# Save LoRA adapter (only 6-8MB!)
# model.save_pretrained("./my-lora-adapter")

In [31]:
# Later: load adapter
# from peft import PeftModel
# base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3-8B")
# model = PeftModel.from_pretrained(base_model, "./my-lora-adapter")

In [32]:
# Generate with fine-tuned model
# prompt = "### Instruction:\nSummarize this text:\n\n### Input:\n[Your text]\n\n### Response:\n"
# inputs = tokenizer(prompt, return_tensors="pt")
# outputs = model.generate(**inputs, max_new_tokens=256)
# print(tokenizer.decode(outputs[0]))

# Tinker

In [23]:
import tinker


In [ ]:
# create a ServiceInterface. 
# This lets you find out what base models are available to be fine-tuned.

service_client = tinker.ServiceClient()
print("Available models:")
for item in service_client.get_server_capabilities().supported_models:
    print("- " + item.model_name)

Available models:
- deepseek-ai/DeepSeek-V3.1
- deepseek-ai/DeepSeek-V3.1-Base
- moonshotai/Kimi-K2-Thinking
- meta-llama/Llama-3.1-70B
- meta-llama/Llama-3.1-8B
- meta-llama/Llama-3.1-8B-Instruct
- meta-llama/Llama-3.2-1B
- meta-llama/Llama-3.2-3B
- meta-llama/Llama-3.3-70B-Instruct
- Qwen/Qwen3-235B-A22B-Instruct-2507
- Qwen/Qwen3-30B-A3B
- Qwen/Qwen3-30B-A3B-Base
- Qwen/Qwen3-30B-A3B-Instruct-2507
- Qwen/Qwen3-32B
- Qwen/Qwen3-4B-Instruct-2507
- Qwen/Qwen3-8B
- Qwen/Qwen3-8B-Base
- Qwen/Qwen3-VL-235B-A22B-Instruct
- Qwen/Qwen3-VL-30B-A3B-Instruct
- openai/gpt-oss-120b
- openai/gpt-oss-20b


Session heartbeat failed for 132.7971683749929 seconds for session ca4d2883-0e52-58f4-a347-8cfa746807c6. Last exception: APITimeoutError: Request timed out..
Your connection may be unreliable or Tinker is down. If this persists, the session will be terminated.
Session heartbeat failed for 152.80325712496415 seconds for session ca4d2883-0e52-58f4-a347-8cfa746807c6. Last exception: APITimeoutError: Request timed out..
Your connection may be unreliable or Tinker is down. If this persists, the session will be terminated.
Session heartbeat failed for 181.24661608296447 seconds for session ca4d2883-0e52-58f4-a347-8cfa746807c6. Last exception: APITimeoutError: Request timed out..
Your connection may be unreliable or Tinker is down. If this persists, the session will be terminated.
Session heartbeat failed for 201.25373570795637 seconds for session ca4d2883-0e52-58f4-a347-8cfa746807c6. Last exception: APITimeoutError: Request timed out..
Your connection may be unreliable or Tinker is down. If 

In [25]:
base_model = "meta-llama/Llama-3.1-8B"
training_client = service_client.create_lora_training_client(
    base_model=base_model
)

Supervised learning: In the context of language model fine-tuning, this means minimizing a weighted cross-entropy loss on token sequences---equivalently, maximizing the log-probability of the specified target tokens.

There are a few ways that SL is commonly used in LLM fine-tuning pipelines:
- Instruction tuning: This is the first step in post-training pipelines, applied to the base (raw, pretrained) model. Typically, we do SL on a high-quality dataset that demonstrates the correct format and style, while boosting the model's reasoning and instruction-following.
- Context distillation / prompt distillation: let's say we have a generic model that can do chat / instruction following / reasoning, but we want to adjust how it behaves in a certain scenario. We can add some instructions to the system message of our model. However, the system message might grow impractically long and start ignoring some of its instructions. So it's often better to create a supervised dataset on a narrow prompt distribution, with a shorter set of instructions that that are targeted at these prompts.

### Tinker SFT on our `training_data.json`

This section converts `data/finetuning/training_data.json` (instruction/input/output) into Tinker Cookbook's conversation JSONL format, then runs the built-in supervised fine-tuning loop.

Prereqs:
- `TINKER_API_KEY` must be set in your environment (dotenv is already loaded at the top of this notebook).
- `tinker-cookbook` repo must be present at `../tinker-cookbook` (relative to this notebook's working dir).


In [26]:
import sys
from pathlib import Path

print("cwd:", Path.cwd())

# Make `tinker_cookbook` importable from this notebook without a pip install.
# We search upward for a directory that contains `tinker_cookbook/__init__.py`.

def find_tinker_cookbook_repo(start: Path) -> Path:
    candidates: list[Path] = []
    for p in [start, *start.parents]:
        # if we're in ar_finetune/, the sibling repo is often here:
        candidates.append(p / "tinker-cookbook")
        # or the repo root itself could be the current dir:
        candidates.append(p)

    for repo in candidates:
        if (repo / "tinker_cookbook" / "__init__.py").exists():
            return repo

    raise FileNotFoundError(
        "Could not find the tinker-cookbook repo (missing tinker_cookbook/__init__.py). "
        "Expected it at ../tinker-cookbook relative to ar_finetune, or somewhere above your cwd."
    )

TINKER_COOKBOOK_REPO = find_tinker_cookbook_repo(Path.cwd())
if str(TINKER_COOKBOOK_REPO) not in sys.path:
    sys.path.insert(0, str(TINKER_COOKBOOK_REPO))

import tinker_cookbook  # noqa: F401

print("tinker-cookbook repo:", TINKER_COOKBOOK_REPO)
print("tinker_cookbook import ok")


cwd: /Users/ext-elias.melas/Documents/Gitcode/ar_finetune
tinker-cookbook repo: /Users/ext-elias.melas/Documents/Gitcode/tinker-cookbook
tinker_cookbook import ok


In [27]:
import json

print("cwd:", Path.cwd())

# Robustly locate the repo root even if you `cd`'d somewhere else in the notebook.
# We search upward for the file we need.

def find_ar_finetune_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        candidate = p / "data/finetuning/training_data.json"
        if candidate.exists():
            return p
    raise FileNotFoundError(
        "Could not find data/finetuning/training_data.json by searching upward from cwd. "
        "Either run `%cd ..` to the ar_finetune repo root, or set AR_FINETUNE_ROOT manually."
    )

AR_FINETUNE_ROOT = find_ar_finetune_root(Path.cwd())
print("ar_finetune root:", AR_FINETUNE_ROOT)

in_path = AR_FINETUNE_ROOT / "data/finetuning/training_data.json"
out_path = AR_FINETUNE_ROOT / "data/finetuning/tinker_conversations.jsonl"

data = json.loads(in_path.read_text(encoding="utf-8"))
assert isinstance(data, list)

out_path.parent.mkdir(parents=True, exist_ok=True)
with out_path.open("w", encoding="utf-8") as f:
    for ex in data:
        instruction = (ex.get("instruction") or "").strip()
        user_input = (ex.get("input") or "").strip()
        output = (ex.get("output") or "").strip()

        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": user_input},
            {"role": "assistant", "content": output},
        ]
        f.write(json.dumps({"messages": messages}, ensure_ascii=False) + "\n")

print("Wrote:", out_path)
print("Head:")
print(out_path.open("r", encoding="utf-8").read().splitlines()[0][:500])


cwd: /Users/ext-elias.melas/Documents/Gitcode/ar_finetune
ar_finetune root: /Users/ext-elias.melas/Documents/Gitcode/ar_finetune
Wrote: /Users/ext-elias.melas/Documents/Gitcode/ar_finetune/data/finetuning/tinker_conversations.jsonl
Head:
{"messages": [{"role": "system", "content": "You are a support engineer in a mobile gaming company. Your task is to classify if a chat log belongs to a phishing attempt or not.\n\nIf the chat log is a phishing attempt, return \"PHISHING\" for category, the reason for the phishing attempt, the reason keyword and the source.\nIf the chat log is not a phishing attempt, return \"GENUINE\" for category, an empty string for reason, an empty string for reason_keyword and an empty list for source.\nIf u


In [28]:
import chz
import tinker

from tinker_cookbook import model_info
from tinker_cookbook.renderers import TrainOnWhat
from tinker_cookbook.supervised import train
from tinker_cookbook.supervised.data import FromConversationFileBuilder
from tinker_cookbook.supervised.types import ChatDatasetBuilderCommonConfig

# Pick a base model available on Tinker (you printed the list above).
model_name = "meta-llama/Llama-3.1-8B"
renderer_name = model_info.get_recommended_renderer_name(model_name)

common_config = ChatDatasetBuilderCommonConfig(
    model_name_for_tokenizer=model_name,
    renderer_name=renderer_name,
    max_length=8192,      # start here; increase if you want the full long prompts
    batch_size=16,
    train_on_what=TrainOnWhat.ALL_ASSISTANT_MESSAGES,
)

dataset_builder = FromConversationFileBuilder(
    common_config=common_config,
    file_path=str(out_path.resolve()),
    test_size=128,
    shuffle_seed=0,
)

config = train.Config(
    log_path=str(Path("/tmp/tinker-examples/sl_ar_phishing").expanduser()),
    model_name=model_name,
    dataset_builder=dataset_builder,
    learning_rate=2e-4,
    lr_schedule="linear",
    num_epochs=1,
    eval_every=10,
    save_every=20,
)

print("Renderer:", renderer_name)
print("Dataset:", out_path.resolve())
print("Log path:", config.log_path)
print("(Training is async; run the next cell to start.)")


/Users/ext-elias.melas/Documents/Gitcode/ar_finetune/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Renderer: role_colon
Dataset: /Users/ext-elias.melas/Documents/Gitcode/ar_finetune/data/finetuning/tinker_conversations.jsonl
Log path: /tmp/tinker-examples/sl_ar_phishing
(Training is async; run the next cell to start.)


In [ ]:
# Start training
# If you're in a notebook where an event loop is already running, you can use `await`.
# await train.main(config)


tinker_cookbook.checkpoint_utils:21 [INFO] No checkpoints found at /tmp/tinker-examples/sl_ar_phishing/checkpoints.jsonl
tinker_cookbook.checkpoint_utils:52 [INFO] No checkpoints found with key state_path in /tmp/tinker-examples/sl_ar_phishing
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabl

tinker_cookbook.utils.ml_log:195 [INFO] 
Configuration:
  log_path: '/tmp/tinker-examples/sl_ar_phishing'
  model_name: 'meta-llama/Llama-3.1-8B'
  load_checkpoint_path: None
  dataset_builder: {'common_config': {'model_name_for_tokenizer': 'meta-llama/Llama-3.1-8B', 'renderer_name': 
'role_colon', 'max_length': 8192, 'ba ... 
/ext-elias.melas/Documents/Gitcode/ar_finetune/data/finetuning/tinker_conversations.jsonl', 'test_size': 128, 
'shuffle_seed': 0}
  learning_rate: 0.0002
  lr_schedule: 'linear'
  num_epochs: 1
  lora_rank: 32
  base_url: None
  evaluator_builders: []
  infrequent_evaluator_builders: []
  save_every: 20
  eval_every: 10
  infrequent_eval_every: 100
  adam_beta1: 0.9
  adam_beta2: 0.95
  adam_eps: 1e-08
  wandb_project: None
  wandb_name: None
  enable_trace: False
tinker_cookbook.utils.ml_log:475 [INFO] Logging to: /tmp/tinker-examples/sl_ar_phishing
tinker.lib.public_interfaces.service_client:66 [INFO] ServiceClient initialized for session 534e2ca0-8fc8-5613-b49

tinker_cookbook.utils.ml_log:195 [INFO] 
                    Step 0                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000200   │
│ num_loss_tokens                │ 128.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 51443      │
│ progress                       │ 0.006803   │
│ test/nll                       │ 2.952745   │
│ time/evals                     │ 12.881727  │
│ time/get_batch                 │ 0.057795   │
│ time/step                      │ 5.179030   │
│ time/total                     │ 18.203809  │
│ train_mean_nll                 │ 2.954427   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.06 seconds
tinker_coo

tinker_cookbook.utils.ml_log:195 [INFO] 
                    Step 1                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000199   │
│ num_loss_tokens                │ 128.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 53850      │
│ progress                       │ 0.013605   │
│ time/get_batch                 │ 0.066057   │
│ time/step                      │ 1.545786   │
│ time/total                     │ 6.893175   │
│ train_mean_nll                 │ 1.725318   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.06 seconds
tinker_cookbook.supervised.train:259 [INFO] <|begin_of_text|>System: You are a support engineer in a mobil

tinker_cookbook.utils.ml_log:195 [INFO] 
                    Step 2                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000197   │
│ num_loss_tokens                │ 128.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 55431      │
│ progress                       │ 0.020408   │
│ time/get_batch                 │ 0.064507   │
│ time/step                      │ 1.585399   │
│ time/total                     │ 3.286883   │
│ train_mean_nll                 │ 0.747786   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.07 seconds
tinker_cookbook.supervised.train:259 [INFO] <|begin_of_text|>System: You are a support engineer in a mobil

...

tinker_cookbook.utils.ml_log:195 [INFO] 
                   Step 142                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000007   │
│ num_loss_tokens                │ 128.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 56639      │
│ progress                       │ 0.972789   │
│ time/get_batch                 │ 0.067259   │
│ time/step                      │ 4.273555   │
│ time/total                     │ 4.438233   │
│ train_mean_nll                 │ 0.053256   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.07 seconds
tinker_cookbook.supervised.train:259 [INFO] <|begin_of_text|>System: You are a support engineer in a mobil

tinker_cookbook.utils.ml_log:195 [INFO] 
                   Step 143                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000005   │
│ num_loss_tokens                │ 128.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 55946      │
│ progress                       │ 0.979592   │
│ time/get_batch                 │ 0.070332   │
│ time/step                      │ 1.730106   │
│ time/total                     │ 6.169226   │
│ train_mean_nll                 │ 0.023978   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.07 seconds
tinker_cookbook.supervised.train:259 [INFO] <|begin_of_text|>System: You are a support engineer in a mobil

tinker_cookbook.utils.ml_log:195 [INFO] 
                   Step 144                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000004   │
│ num_loss_tokens                │ 128.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 53401      │
│ progress                       │ 0.986395   │
│ time/get_batch                 │ 0.070213   │
│ time/step                      │ 1.020035   │
│ time/total                     │ 2.922471   │
│ train_mean_nll                 │ 0.057087   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.07 seconds
tinker_cookbook.supervised.train:259 [INFO] <|begin_of_text|>System: You are a support engineer in a mobil

tinker_cookbook.utils.ml_log:195 [INFO] 
                   Step 145                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000003   │
│ num_loss_tokens                │ 128.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 51539      │
│ progress                       │ 0.993197   │
│ time/get_batch                 │ 0.068627   │
│ time/step                      │ 1.999471   │
│ time/total                     │ 3.189792   │
│ train_mean_nll                 │ 0.057794   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting step
tinker_cookbook.utils.misc_utils:23 [INFO] step took 2.13 seconds
tinker_cookbook.utils.ml_log:143 [INFO] Wrote metrics to /tmp/tinker-examples/sl_ar_phishing/metrics.jsonl


tinker_cookbook.utils.ml_log:195 [INFO] 
                   Step 146                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000001   │
│ num_loss_tokens                │ 128.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 50098      │
│ progress                       │ 1.000000   │
│ time/get_batch                 │ 0.068921   │
│ time/step                      │ 2.132003   │
│ time/total                     │ 4.221624   │
│ train_mean_nll                 │ 0.052104   │
└────────────────────────────────┴────────────┘
tinker_cookbook.checkpoint_utils:81 [INFO] Saved checkpoints: {'state_path': 'tinker://534e2ca0-8fc8-5613-b494-3062bc9510ef:train:0/weights/final', 'sampler_path': 'tinker://534e2ca0-8fc8-5613-b494-3062bc9510ef:train:0/sampler_weights/fina

In [44]:
# If you ever need to resume, rerun the config cell and then rerun training.
# The loop will detect checkpoints in `log_path` and resume automatically.



In [83]:
# Save a final checkpoint explicitly (and print the persistent Tinker paths).
# (The training loop already saves periodically + at the end, but this makes it obvious.)

service_client = tinker.ServiceClient()

# `train.main` creates its own training client internally, so we save via the run logs:
# We'll grab the last recorded sampler/state paths from checkpoints.jsonl.
from tinker_cookbook.checkpoint_utils import get_last_checkpoint

ckpt_state = get_last_checkpoint(config.log_path, required_key="state_path")
ckpt_sampler = get_last_checkpoint(config.log_path, required_key="sampler_path")

print("Last state checkpoint:", ckpt_state.get("state_path") if ckpt_state else None)
print("Last sampler checkpoint:", ckpt_sampler.get("sampler_path") if ckpt_sampler else None)

# Create a SamplingClient you can use for inference:
if ckpt_sampler:
    sampling_client = service_client.create_sampling_client(model_path=ckpt_sampler["sampler_path"])
    print("Sampling client ready:", sampling_client)
else:
    print("No sampler checkpoint found yet — make sure training finished and wrote checkpoints.jsonl.")


tinker.lib.public_interfaces.service_client:66 [INFO] ServiceClient initialized for session 6ac8f598-028c-5540-acd4-df06b18be470
tinker_cookbook.checkpoint_utils:24 [INFO] Reading checkpoints from /tmp/tinker-examples/sl_ar_phishing/checkpoints.jsonl
tinker_cookbook.checkpoint_utils:46 [INFO] Found 9 valid checkpoints with key 'state_path' in /tmp/tinker-examples/sl_ar_phishing
tinker_cookbook.checkpoint_utils:49 [INFO] Using last checkpoint: {'name': 'final', 'epoch': 1, 'batch': 147, 'state_path': 'tinker://9e2c678a-ef0f-5088-acae-7d62e8bfa5ee:train:0/weights/final', 'sampler_path': 'tinker://9e2c678a-ef0f-5088-acae-7d62e8bfa5ee:train:0/sampler_weights/final'}
tinker_cookbook.checkpoint_utils:24 [INFO] Reading checkpoints from /tmp/tinker-examples/sl_ar_phishing/checkpoints.jsonl
tinker_cookbook.checkpoint_utils:46 [INFO] Found 9 valid checkpoints with key 'sampler_path' in /tmp/tinker-examples/sl_ar_phishing
tinker_cookbook.checkpoint_utils:49 [INFO] Using last checkpoint: {'name': 

Last state checkpoint: tinker://9e2c678a-ef0f-5088-acae-7d62e8bfa5ee:train:0/weights/final
Last sampler checkpoint: tinker://9e2c678a-ef0f-5088-acae-7d62e8bfa5ee:train:0/sampler_weights/final
Sampling client ready: <tinker.SamplingClient object at 0x570e5efd0>


tinker.lib.internal_client_holder:202 [WARNING] Session heartbeat failed for 120.07031120899956 seconds for session 6ac8f598-028c-5540-acd4-df06b18be470. Last exception: APIConnectionError: Connection error..
Your connection may be unreliable or Tinker is down. If this persists, the session will be terminated.
tinker.lib.internal_client_holder:202 [WARNING] Session heartbeat failed for 130.0760777919986 seconds for session 6ac8f598-028c-5540-acd4-df06b18be470. Last exception: APIConnectionError: Connection error..
Your connection may be unreliable or Tinker is down. If this persists, the session will be terminated.
tinker.lib.internal_client_holder:202 [WARNING] Session heartbeat failed for 120.05757633300163 seconds for session 6ac8f598-028c-5540-acd4-df06b18be470. Last exception: APIConnectionError: Connection error..
Your connection may be unreliable or Tinker is down. If this persists, the session will be terminated.


In [31]:
# Optional: download the LoRA adapter weights to a local tar archive.
# Requires the sampler checkpoint path from the previous cell.

import urllib.request

if ckpt_sampler is None:
    raise RuntimeError("No sampler checkpoint available to download.")

sampler_path = ckpt_sampler["sampler_path"]
rest_client = service_client.create_rest_client()
url_resp = rest_client.get_checkpoint_archive_url_from_tinker_path(sampler_path).result()

local_tar = Path("./tinker_adapter_archive.tar").resolve()
urllib.request.urlretrieve(url_resp.url, str(local_tar))
print("Downloaded to:", local_tar)


Downloaded to: /Users/ext-elias.melas/Documents/Gitcode/ar_finetune/tinker_adapter_archive.tar


In [84]:
# # Local inference helper (no chat_template needed)
# # This matches the exact formatting you used when creating `dataset["text"]` in the notebook.

# import re
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from peft import PeftModel

# BASE_MODEL_PATH = "./models/llama-3.1-8b"  # or HF model id
# ADAPTER_DIR = "./lora_adapter"            # directory you extracted the tar into

# def build_sft_prompt(instruction: str, user_input: str) -> str:
#     prompt = f"### Instruction:\n{instruction.strip()}\n\n"
#     if user_input.strip():
#         prompt += f"### Input:\n{user_input.strip()}\n\n"
#     prompt += "### Response:\n"
#     return prompt

# @torch.inference_mode()
# def classify_with_lora(instruction: str, user_input: str, max_new_tokens: int = 32) -> tuple[str, bool | None]:
#     device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

#     tok = AutoTokenizer.from_pretrained(BASE_MODEL_PATH)
#     tok.pad_token = tok.eos_token

#     model = AutoModelForCausalLM.from_pretrained(
#         BASE_MODEL_PATH,
#         torch_dtype=torch.float16 if device in {"cuda", "mps"} else torch.float32,
#     )
#     model = PeftModel.from_pretrained(model, ADAPTER_DIR)
#     model.eval().to(device)

#     prompt = build_sft_prompt(instruction, user_input)
#     inputs = tok(prompt, return_tensors="pt").to(device)

#     out = model.generate(
#         **inputs,
#         max_new_tokens=max_new_tokens,
#         do_sample=False,
#         temperature=0.0,
#     )
#     gen = tok.decode(out[0][inputs["input_ids"].shape[-1] :], skip_special_tokens=True).strip()

#     # Your training targets look like: `is_phishing: True/False`
#     m = re.search(r"is_phishing\s*:\s*(True|False)", gen)
#     parsed = None if m is None else (m.group(1) == "True")
#     return gen, parsed

# # Example usage (reuse your existing system_prompt + a user_prompt):
# gen_text, is_phish = classify_with_lora(system_prompt, user_prompt)
# print(gen_text, is_phish)


## Using this adapter

In [227]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from peft import PeftModel

# base_model_path = "./models/llama-3.1-8b"  # or HF id if you have access
# adapter_path = "./lora_adapter"

# tokenizer = AutoTokenizer.from_pretrained(base_model_path)
# model = AutoModelForCausalLM.from_pretrained(
#     base_model_path,
#     torch_dtype=torch.float16,
# )

# model = PeftModel.from_pretrained(model, adapter_path)
# model.eval()

# device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
# model.to(device)

# system_prompt = open("data/verification/system_prompt.md", "r").read()
# user_prompt = "... build the same user_prompt you used before ..."

# messages = [
#     {"role": "system", "content": system_prompt},
#     {"role": "user", "content": user_prompt},
# ]

# prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# inputs = tokenizer(prompt, return_tensors="pt").to(device)

# out = model.generate(**inputs, max_new_tokens=64, do_sample=False, temperature=0.0)
# pred = tokenizer.decode(out[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True).strip()
# print(pred)

In [85]:
# Use the fine-tuned Tinker model (SamplingClient) to classify rows like `process_all_messages_phishing`,

from tinker_cookbook.completers import TinkerMessageCompleter
from tinker_cookbook import renderers as cookbook_renderers
from tinker_cookbook.tokenizer_utils import get_tokenizer

# Build a renderer that matches training (you used role_colon during training).
ft_model_name = config.model_name
ft_renderer_name = renderer_name

_tok = get_tokenizer(ft_model_name)
_renderer = cookbook_renderers.get_renderer(ft_renderer_name, _tok)

# Deterministic classification: temperature=0 and small max_tokens.
tinker_completer = TinkerMessageCompleter(
    sampling_client=sampling_client,
    renderer=_renderer,
    max_tokens=32,
)

print("FT model:", ft_model_name)
print("Renderer:", ft_renderer_name)
print("Stop:", _renderer.get_stop_sequences())


FT model: meta-llama/Llama-3.1-8B
Renderer: role_colon
Stop: ['\n\nUser:']


In [86]:
import re
import asyncio
from tqdm.auto import tqdm


def _build_user_prompt(messages: str, features: str, ml_features: str) -> str:
    return f"""
    Message Chat log:
    ---------------------------------------------------
    \n{messages}\n\n
    ---------------------------------------------------
    Compromised account analysis:
    ---------------------------------------------------
    \n{features}\n\n
    ---------------------------------------------------

    Machine learning features:
    ---------------------------------------------------
    \n{ml_features}\n\n
    ---------------------------------------------------

    """


async def classify_phishing_tinker(
    messages: str,
    features: str,
    ml_features: str,
    *,
    system_prompt: str = system_prompt,
) -> tuple[VerificationClassification, str]:
    user_prompt = _build_user_prompt(messages, features, ml_features)

    convo = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    assistant_msg = await tinker_completer(convo)
    # print(assistant_msg)
    text = str(assistant_msg.get("content", "")).strip()

    # Example content:
    #   "is_phishing: True, agent_notes: ..."
    m_notes = re.search(r"agent_notes\s*:\s*(.*)", text, flags=re.IGNORECASE)
    agent_notes = m_notes.group(1).strip() if m_notes else ""

    m = re.search(r"is_phishing\s*:\s*(True|False)", text)
    if m is None:
        # Fallback: treat unparseable output as ASSIGN_TO_AGENT
        return (
            VerificationClassification(
                category=VerificationCategory.ASSIGN_TO_AGENT,
                reason="",
                reason_keyword="",
                source=[],
            ),
            agent_notes,
        )

    is_phishing = m.group(1) == "True"
    category = VerificationCategory.PHISHING if is_phishing else VerificationCategory.GENUINE
    return (VerificationClassification(category=category, reason="", reason_keyword="", source=[]), agent_notes)


async def process_all_messages_phishing_tinker(
    df_messages: pd.DataFrame,
    *,
    concurrency: int = 16,
    max_rows: int | None = None,
) -> List[Dict[str, Any]]:
    """Like `process_all_messages_phishing`, but uses the fine-tuned Tinker model instead of GPT."""

    sem = asyncio.Semaphore(concurrency)

    rows = list(df_messages.iterrows())
    if max_rows is not None:
        rows = rows[:max_rows]

    async def process_single(row: pd.Series) -> Dict[str, Any]:
        async with sem:
            intent_result, agent_notes = await classify_phishing_tinker(
                messages=row["messages_player"],
                features=row["features"],
                ml_features=row["ml_features"],
            )
            return {
                "ticket_id": row["ticket_id"],
                "message": row["messages_player"],
                "ticket_tags": row["ticket_tags_with_description"],
                "features": row["features"],
                "ml_features": row["ml_features"],
                "category": intent_result.category,
                "reason": intent_result.reason,
                "reason_keyword": intent_result.reason_keyword,
                "source": intent_result.source,
                "agent_notes_pred": agent_notes,
                "success": True,
                "error": None,
            }

    tasks = [asyncio.create_task(process_single(row)) for _, row in rows]

    results: list[Dict[str, Any]] = []
    for fut in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
        results.append(await fut)

    return results


# Example:
tinker_results = await process_all_messages_phishing_tinker(lora_test, max_rows=None)
df_tinker = pd.DataFrame(tinker_results)

100%|██████████| 624/624 [01:26<00:00,  7.18it/s]


In [87]:
df_phishing_messages_tinker = lora_test.merge(
    df_tinker[["ticket_id", "category", "agent_notes_pred"]],
    on="ticket_id",
    how="left",
)
df_phishing_messages_tinker["is_phishing_llm"] = df_phishing_messages_tinker.apply(create_llm_label, axis=1)
df_phishing_messages_tinker["category"].value_counts(dropna=False)

category
PHISHING           360
GENUINE            248
ASSIGN_TO_AGENT     16
Name: count, dtype: int64

In [89]:
y_true = df_phishing_messages_tinker["is_phishing"]
y_pred = df_phishing_messages_tinker["is_phishing_llm"]

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
# [[TN, FP],
#  [FN, TP]]

print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
print(f"Confusion Matrix: {cm}")

Precision: 0.5053, Recall: 0.6090, F1 Score: 0.5523
Confusion Matrix: [[126 186]
 [122 190]]


In [90]:
for i,row in df_phishing_messages_tinker.iloc[1:2].iterrows():
    print(row['agent_notes'])
    print(row['is_phishing'])
    print(row['category'])
    print(row['agent_notes_pred'])

1. LA recovery RA = Clean | Unprotected | Unlocked | SCID Connected | Eligible TA = Clean | Unprotected | Unlocked | SCID Not Connected | Eligible https://pandora.supercellsupport.com/clashofclans/accounts/47-9898869 TH Level: TH 12 => TH 13 Young Player: No | No Clan Leader: No | No Languages: PH/EN/EN-PH | PH/EN/EN-US Online: 3 hours, 17 minutes and 33 seconds ago on 10/09/2025 05:51 - 05:52 AM | 2 years, 9 months, 14 days, 13 hours, 6 minutes and 9 seconds ago on 25/11/2022 20:02 Matching Devices: ID Match, Oppo CPH1803RU, Huawei Honor 7S, Vivo Y53, Myphone my85 DTV, ASUS_X013D, MyPhone A919 Duo Matching IP: No High Value Account: No | No Requires Special Handling: No | No Risk Level: none Clean Accounts: Yes | Yes Has transactions: No | No Support Notes: no Other issues from requester: no Other claimants: no Phishing bans on related accounts: no Notable account: no Note(s) about devices: no HS/Pandora countries: match  IP Addresses: / Eligibility Check for RA and/or TA: no previous